In [1]:
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
import requests
import os
import urllib.request
from urllib.request import urlretrieve

In [36]:
count = 1
url_list = []
while count < 1300:
    url_test = 'https://www.sec.gov/cgi-bin/srch-edgar?text=S-1&start=' + str(count) + '&count=80&first=2018&last=2019'
    response = requests.get(url_test)
    if response.status_code == 200:
        url_list.append(url_test)
        count += 80


In [55]:
date_list = []
td_list = []
hyperlink_text = []

hyperlink_dict = {}

for url in url_list:
    response = requests.get(url)
    page = response.text
    soup= BeautifulSoup(page,'lxml')
    for tag in soup.find_all('tr'):
        for j in tag.find_all('td'):
            if ('/2019' in j.text) or ('/2018' in j.text):
                date = j.text
                date = date.replace("/", "-")
                date_list.append(date)
            for k in j.find_all('a'):   
                if ("Search" not in k.text) and ("Home" not in k.text) and ("Previous" not in k.text) and ("text" not in k.text) and ("html" not in k.text) and (k.text != "\n"):
                    key =  date + "_" + k.text # this gets company name and date submitted as key
                    key = key.replace(" ", "_")
                    key = key.replace(".", "")
                    key = key.replace("/", "--")
                    td_list.append(key)

                if k.text == "[text]":
                    value = "https://www.sec.gov" + k['href']
                    hyperlink_text.append(value)

hyperlink_dict = dict(zip(td_list, hyperlink_text))

In [56]:
len(hyperlink_dict)

1239

In [57]:
path = "/Users/kathmbell/Dropbox/project_04"

for key, url in hyperlink_dict.items():
    page = urllib.request.urlopen(url)
    name = key + ".txt"
    completeName = os.path.join(path, name) 
    f = open(completeName,'w')
    content = str(page.read())
    f.write(content)
    f.close()

